In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import nn
from torch.utils.data import DataLoader
from networks import A1

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
train_data= datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor(),)

test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor(),)

In [4]:
labels_map={
    0: 'T-shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot',
}

In [5]:
sample_idx = torch.randint(len(train_data), size = (1,)).item()
image, label = train_data[sample_idx]
image.shape

torch.Size([1, 28, 28])

### La cella sotto va eliminata !!!!!!!!!!!!!!!!!!!!

In [7]:
test_convolutional_layer = nn.Conv2d(in_channels=1, out_channels=4, kernel_size=5) # kernel size 4 = (4,4) = 4 x 4
flatten = nn.Flatten(start_dim=-3)
linear1 = nn.Linear(2304, 10)
softmax = nn.Softmax(dim=0)
seq = nn.Sequential()
# se aggiungessi il parametro padding="same" otterrei l'output della stessa dimensione dell'input


x = train_data[0][0]
print('Shape of x:', x.shape)
## The shape will be 128, 1, 28, 28 because we have 128 images in a batch, 1 channel, and 28x28 pixels

x2 = test_convolutional_layer(x)
print('Shape of x2:', x2.shape)
x3 = flatten(x2)
print('Shape of x3:', x3.shape)
x4 = linear1(x3)
print('Shape of x4:', x4.shape)
x5 = softmax(x4)



print('Shape of x5:', x5.shape)
x5

Shape of x: torch.Size([1, 28, 28])
Shape of x2: torch.Size([4, 24, 24])
Shape of x3: torch.Size([2304])
Shape of x4: torch.Size([10])
Shape of x5: torch.Size([10])


tensor([0.0947, 0.1072, 0.0737, 0.1133, 0.0716, 0.0982, 0.1236, 0.0980, 0.1102,
        0.1094], grad_fn=<SoftmaxBackward0>)

In [ ]:
batch_size = 128

train_dataloader= DataLoader(train_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
def HF_train_loop(dataloader, model, loss_fn, optimizer):
    model.freeze_conv_param()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
    
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 1000 == 0:
          loss, current = loss.item(), (batch + 1) * len(X)
          print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    # torch save model with torch.save()
    torch.save({'model_weights': model.state_dict()}, 'model.pt')

In [ ]:
def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
  return (100*correct)